# Flux Ops Center - SPCS Deployment Notebook

**Purpose**: Deploy Flux Operations Center SPCS infrastructure using Snowflake Notebooks

This notebook deploys:
- Image repository for Docker containers
- Compute pool for SPCS service
- SPCS service with proper configuration
- Snowflake Postgres instance (optional)

**Prerequisites**:
1. flux-utility-solutions deployed (creates database, schemas, tables)
2. Docker image built and pushed to registry

**Estimated Time**: 10-15 minutes

---

## Configuration

Set your deployment parameters:

In [ ]:
# =============================================================================
# CONFIGURATION - Edit these values for your environment
# =============================================================================

# Target database (must exist - deploy flux-utility-solutions first)
DATABASE = "FLUX_DB"
SCHEMA = "PUBLIC"
WAREHOUSE = "FLUX_WH"

# SPCS Configuration
IMAGE_REPO = "FLUX_OPS_CENTER_IMAGES"
COMPUTE_POOL = "FLUX_OPS_CENTER_POOL"
SERVICE_NAME = "FLUX_OPS_CENTER_SERVICE"
IMAGE_TAG = "latest"

# Compute pool sizing
INSTANCE_FAMILY = "CPU_X64_S"  # Options: CPU_X64_XS, CPU_X64_S, CPU_X64_M, CPU_X64_L
MIN_NODES = 1
MAX_NODES = 2

# Postgres Configuration (for map visualization)
POSTGRES_INSTANCE = "FLUX_OPS_POSTGRES"
POSTGRES_HOST = ""  # Will be populated after creation

print(f"Target Database: {DATABASE}")
print(f"Service Name: {SERVICE_NAME}")
print(f"Compute Pool: {COMPUTE_POOL}")

In [ ]:
# Get Snowflake session
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Verify connection
result = session.sql("SELECT CURRENT_USER(), CURRENT_ROLE(), CURRENT_ACCOUNT()").collect()
print(f"User: {result[0][0]}")
print(f"Role: {result[0][1]}")
print(f"Account: {result[0][2]}")

# Set context
session.sql(f"USE DATABASE {DATABASE}").collect()
session.sql(f"USE SCHEMA {SCHEMA}").collect()
session.sql(f"USE WAREHOUSE {WAREHOUSE}").collect()
print(f"\nContext set to: {DATABASE}.{SCHEMA}")

## Phase 1: Verify Prerequisites

Check that flux-utility-solutions has been deployed:

In [ ]:
# Verify required schemas exist (from flux-utility-solutions)
required_schemas = ['PRODUCTION', 'APPLICATIONS', 'ML_DEMO', 'CASCADE_ANALYSIS']

existing_schemas = session.sql(f"""
    SELECT SCHEMA_NAME 
    FROM {DATABASE}.INFORMATION_SCHEMA.SCHEMATA
    WHERE SCHEMA_NAME IN ({','.join([f"'{s}'" for s in required_schemas])})
""").collect()

existing = [row[0] for row in existing_schemas]
missing = [s for s in required_schemas if s not in existing]

print("PREREQUISITE CHECK")
print("=" * 50)
for schema in required_schemas:
    status = "✓" if schema in existing else "✗ MISSING"
    print(f"  {status} {schema}")

if missing:
    print(f"\n⚠️  Missing schemas: {missing}")
    print("   Deploy flux-utility-solutions first, or use standalone quickstart:")
    print("   snow sql -f scripts/sql/00_standalone_quickstart.sql")
else:
    print(f"\n✓ All prerequisites met - ready to deploy SPCS")

## Phase 2: Create Image Repository

In [ ]:
# Create image repository
session.sql(f"""
    CREATE IMAGE REPOSITORY IF NOT EXISTS {IMAGE_REPO}
    COMMENT = 'Image repository for Flux Operations Center containers'
""").collect()

# Get repository URL for Docker push
repo_info = session.sql(f"SHOW IMAGE REPOSITORIES LIKE '{IMAGE_REPO}'").collect()
if repo_info:
    repo_url = repo_info[0]['repository_url']
    print(f"✓ Image Repository: {IMAGE_REPO}")
    print(f"\nDocker Push Commands:")
    print(f"  docker login {repo_url.split('/')[0]}")
    print(f"  docker build -t flux-ops-center:{IMAGE_TAG} -f Dockerfile.spcs .")
    print(f"  docker tag flux-ops-center:{IMAGE_TAG} {repo_url}/flux_ops_center:{IMAGE_TAG}")
    print(f"  docker push {repo_url}/flux_ops_center:{IMAGE_TAG}")

## Phase 3: Create Compute Pool

In [ ]:
# Create compute pool
session.sql(f"""
    CREATE COMPUTE POOL IF NOT EXISTS {COMPUTE_POOL}
    MIN_NODES = {MIN_NODES}
    MAX_NODES = {MAX_NODES}
    INSTANCE_FAMILY = {INSTANCE_FAMILY}
    AUTO_RESUME = TRUE
    AUTO_SUSPEND_SECS = 300
    COMMENT = 'Compute pool for Flux Operations Center'
""").collect()

# Check status
pool_info = session.sql(f"DESCRIBE COMPUTE POOL {COMPUTE_POOL}").collect()
print(f"✓ Compute Pool: {COMPUTE_POOL}")
print(f"  Instance Family: {INSTANCE_FAMILY}")
print(f"  Min/Max Nodes: {MIN_NODES}/{MAX_NODES}")
print(f"  Status: {pool_info[0]['state'] if pool_info else 'CREATED'}")

## Phase 4: Set Up Snowflake Postgres (Optional)

Required for map visualization with PostGIS spatial data:

In [ ]:
# Check if Postgres instance already exists
existing_pg = session.sql(f"SHOW POSTGRES INSTANCES LIKE '{POSTGRES_INSTANCE}'").collect()

if existing_pg:
    POSTGRES_HOST = existing_pg[0]['host']
    print(f"✓ Postgres Instance exists: {POSTGRES_INSTANCE}")
    print(f"  Host: {POSTGRES_HOST}")
else:
    print(f"Postgres instance '{POSTGRES_INSTANCE}' not found.")
    print("\nTo create one, run:")
    print(f"""
CREATE POSTGRES DATABASE {POSTGRES_INSTANCE}
    POSTGRES_ADMIN_PASSWORD = 'YourSecurePassword123!'
    AUTO_SUSPEND_MINS = 30
    COMPUTE_SIZE = 'HIGHMEM_XL'
    STORAGE_SIZE_GB = 100;
""")
    print("\nThen run: SHOW POSTGRES INSTANCES LIKE 'FLUX_OPS_POSTGRES';")
    print("Copy the 'host' value and update POSTGRES_HOST variable above.")

## Phase 5: Create SPCS Service

**Important**: Ensure Docker image is pushed before running this cell!

In [ ]:
# Verify Postgres host is set
if not POSTGRES_HOST:
    print("⚠️  POSTGRES_HOST not set. Map visualization will not work.")
    print("   Set POSTGRES_HOST variable or create a Postgres instance first.")
    POSTGRES_HOST = "placeholder.postgres.snowflake.app"  # Will fail at runtime

# Create service specification
service_spec = f"""
spec:
  containers:
    - name: flux-ops-center
      image: /{DATABASE}/{SCHEMA}/{IMAGE_REPO}/flux_ops_center:{IMAGE_TAG}
      env:
        SNOWFLAKE_DATABASE: {DATABASE}
        SNOWFLAKE_WAREHOUSE: {WAREHOUSE}
        SNOWFLAKE_SCHEMA: PRODUCTION
        APPLICATIONS_SCHEMA: APPLICATIONS
        ML_SCHEMA: ML_DEMO
        CASCADE_SCHEMA: CASCADE_ANALYSIS
        VITE_POSTGRES_HOST: {POSTGRES_HOST}
        VITE_POSTGRES_PORT: "5432"
        VITE_POSTGRES_DATABASE: postgres
      resources:
        requests:
          cpu: 2
          memory: 4Gi
        limits:
          cpu: 4
          memory: 8Gi
  endpoints:
    - name: app
      port: 8080
      public: true
"""

print("Service Specification:")
print(service_spec)

In [ ]:
# Create the SPCS service
# Note: This will fail if Docker image hasn't been pushed yet

try:
    session.sql(f"""
        CREATE SERVICE IF NOT EXISTS {SERVICE_NAME}
        IN COMPUTE POOL {COMPUTE_POOL}
        FROM SPECIFICATION $${service_spec}$$
        QUERY_WAREHOUSE = {WAREHOUSE}
        COMMENT = 'Flux Operations Center - Grid Visualization & GNN Risk Prediction'
    """).collect()
    
    # Grant public access
    session.sql(f"GRANT USAGE ON SERVICE {SERVICE_NAME} TO ROLE PUBLIC").collect()
    
    print(f"✓ Service created: {SERVICE_NAME}")
    print("\nService is starting... This may take 2-5 minutes.")
    
except Exception as e:
    if "does not exist" in str(e).lower() and "image" in str(e).lower():
        print("✗ Docker image not found in repository!")
        print("\nPush the image first:")
        print(f"  docker push {repo_url}/flux_ops_center:{IMAGE_TAG}")
    else:
        print(f"Error creating service: {e}")

## Phase 6: Verify Deployment

In [ ]:
# Check service status
import json

try:
    status_json = session.sql(f"SELECT SYSTEM$GET_SERVICE_STATUS('{SERVICE_NAME}')").collect()[0][0]
    status = json.loads(status_json)
    
    print("SERVICE STATUS")
    print("=" * 50)
    for container in status:
        print(f"  Container: {container.get('containerName', 'unknown')}")
        print(f"  Status: {container.get('status', 'unknown')}")
        print(f"  Message: {container.get('message', 'none')}")
except Exception as e:
    print(f"Service not ready yet: {e}")

In [ ]:
# Get service endpoint URL
try:
    endpoints = session.sql(f"SHOW ENDPOINTS IN SERVICE {SERVICE_NAME}").collect()
    
    print("SERVICE ENDPOINTS")
    print("=" * 50)
    for ep in endpoints:
        print(f"  Endpoint: {ep['name']}")
        print(f"  URL: {ep['ingress_url']}")
        print(f"\n✓ Flux Ops Center is ready!")
        print(f"  Open: {ep['ingress_url']}")
except Exception as e:
    print(f"Endpoints not available yet: {e}")
    print("\nWait 2-5 minutes for service to start, then run this cell again.")

## Next Steps

### Required: Load PostGIS Spatial Data

**Without this step, the map will not display any data.**

```bash
# From your local machine:
python backend/scripts/load_postgis_data.py --service FLUX_OPS_POSTGRES
```

### Optional: Populate Cascade Analysis Tables

```bash
# Step 1: Compute graph centrality metrics
python backend/scripts/compute_graph_centrality.py

# Step 2: Pre-compute cascade scenarios  
python backend/scripts/cascade_simulator.py --scenarios 100

# Step 3: Train GNN model
python backend/scripts/train_gnn_model.py
```

### Troubleshooting

```sql
-- View service logs
CALL SYSTEM$GET_SERVICE_LOGS('FLUX_OPS_CENTER_SERVICE', '0', 'flux-ops-center', 100);

-- Restart service
ALTER SERVICE FLUX_OPS_CENTER_SERVICE SUSPEND;
ALTER SERVICE FLUX_OPS_CENTER_SERVICE RESUME;
```